In [ ]:
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="LangSmith now uses UUID v7", 
    category=UserWarning,
)

# 7. Unified Production Agent
# From-Scratch → Enhanced → Persistent

## What This Notebook Does

This notebook **unifies** everything from Notebooks 4, 5, and 6 into a single, 
production-ready Deep Agent with Amazon Nova (Bedrock):

| Notebook | What it added | Status here |
|----------|--------------|-------------|
| **NB4** (Full Agent) | Base agent with todos, files, sub-agents | ✅ Foundation |
| **NB5** (Enhanced) | `edit_file`, `glob_files`, `grep_files`, Skills system | ✅ Merged |
| **NB6** (Production) | Prompt caching, DynamoDB persistence | ✅ Merged |

### What was fixed from NB5 + NB6 contradictions:

| Issue | Resolution |
|-------|-----------|
| NB5 uses in-memory `write_todos` / NB6 uses `dynamo_write_todos` | → **Unified**: DynamoDB tools for todos and basic files |
| NB5 uses raw f-string prompt / NB6 uses `build_cached_prompt()` | → **Unified**: Cached prompt with optimal ordering |
| `edit_file`/`glob_files`/`grep_files` operate on in-memory only | → **Documented**: These operate on the in-memory `state["files"]` which is synced via `dynamo_write_file` |
| Agent built twice (once in each notebook) | → **Unified**: Single `create_agent()` call with checkpointer |
| Same test query duplicated | → **Unified**: Single multi-turn test |

---

## Part 1: Architecture Map

### 🏗️ Agents

| Agent | Role | Model | Tools |
|-------|------|-------|-------|
| **Orchestrator** | Coordinates all work, talks to user | Nova Lite/Pro | All tools below |
| **Research Sub-agent** | Web research with context isolation | Nova Lite | `tavily_search`, `think_tool` |

### 🔧 Tools by Category

| Category | Tool | Backend | Description |
|----------|------|---------|-------------|
| **Planning** | `dynamo_write_todos` | DynamoDB | Create/update TODO list (persistent) |
| **Planning** | `dynamo_read_todos` | DynamoDB | Read current TODO list (persistent) |
| **Files (Persistent)** | `dynamo_ls` | DynamoDB + Memory | List all files from both sources |
| **Files (Persistent)** | `dynamo_read_file` | DynamoDB + Memory | Read file (memory-first, DynamoDB fallback) |
| **Files (Persistent)** | `dynamo_write_file` | DynamoDB + Memory | Create/overwrite file (dual-write) |
| **Files (Enhanced)** | `edit_file` | Memory | Find-and-replace in existing files |
| **Files (Enhanced)** | `glob_files` | Memory | Find files by pattern |
| **Files (Enhanced)** | `grep_files` | Memory | Search text across all files |
| **Skills** | `load_skill` | Memory | Load a SKILL.md with detailed instructions |
| **Research** | `tavily_search` | Web API | Web search + save results to files |
| **Research** | `think_tool` | None | Strategic reflection and planning |
| **Delegation** | `task` | — | Spawn isolated sub-agent for complex tasks |

### 📁 State & Persistence

```
┌─────────────────────────────────────────────────────────┐
│                    Agent State (RAM)                      │
│  messages: list[Message]  ← Chat history                 │
│  todos: list[Todo]        ← Task planning                │
│  files: dict[str, str]    ← Virtual filesystem           │
└────────────┬───────────────────────┬────────────────────┘
             │ Checkpointer         │ dynamo_* tools
             ▼                       ▼
┌────────────────────┐  ┌────────────────────────────────┐
│ DeepAgents_State   │  │ DeepAgents_Artifacts           │
│ (DynamoDB)         │  │ (DynamoDB)                     │
│                    │  │                                │
│ PK: thread_id      │  │ PK: thread_id                  │
│ SK: checkpoint_id   │  │ SK: artifact_id                │
│                    │  │     FILE#main.py               │
│ Persists: messages,│  │     TODO#LIST                  │
│ channel values     │  │                                │
└────────────────────┘  └────────────────────────────────┘
```

---

## Part 2: What are Skills? (SKILL.md)

**Skills** are filesystem-based instruction sets for extending agent capabilities without adding more tools.

### How they work (Progressive Disclosure):

1. Agent sees only **name + description** in its system prompt (saves tokens)
2. Agent calls `load_skill("web-research")` when it decides the skill is relevant
3. Full instructions load on demand

### SKILL.md Format:

```yaml
---
name: web-research
description: Use this skill for research tasks requiring web searches.
---

# Instructions
1. Plan your research queries
2. Execute searches with tavily_search
3. Reflect after each search using think_tool
4. Synthesize and deliver findings
```

> **Note:** The `deepagents` library doesn't include skills in its installed version.
> This is our from-scratch implementation.

---

## Part 3: Prompt Caching for Nova

Amazon Nova supports **prompt caching** — the system prompt is cached after Turn 1, 
so subsequent turns only pay **10% of the input cost** for the cached prefix.

### Key Rule: Prompt Ordering

```
┌──────────────────────────────────────────┐
│  STATIC CONTENT (cached, pay 10%)        │
│  ├── Base instructions (persona, rules)  │
│  ├── Tool usage rules                    │
│  └── Skills listing                      │
├──────────── cachePoint ──────────────────│
│  DYNAMIC CONTENT (re-processed, 100%)    │
│  ├── Today's date                        │
│  └── Session-specific context            │
└──────────────────────────────────────────┘
```

`ChatBedrockConverse.create_cache_point()` → `{'cachePoint': {'type': 'default'}}` in `SystemMessage.additional_kwargs`

In [ ]:
from langchain_aws import ChatBedrockConverse
from deep_agents_from_scratch.prompt_caching import (
    create_cached_system_message,
    build_cached_prompt,
    estimate_cache_savings,
)

# --- How the cache point works ---
cache_point = ChatBedrockConverse.create_cache_point()
print("Cache point structure:", cache_point)

# --- Cost savings estimate (5000-token system prompt, 10 turns) ---
savings = estimate_cache_savings(prompt_tokens=5000, turns_per_session=10)
print(f"\n📊 Cache Savings:")
print(f"   Without cache: ${savings['uncached_cost']:.4f}")
print(f"   With cache:    ${savings['cached_cost']:.4f}")
print(f"   Savings:       ${savings['savings']:.4f} ({savings['savings_percent']}%)")

---

## Part 4: DynamoDB Setup

### Tables

| Table | PK | SK | Purpose |
|-------|----|----|--------|
| `DeepAgents_State` | `pk` (thread_id) | `sk` (checkpoint_id) | LangGraph state (chat history, channel values) |
| `DeepAgents_Artifacts` | `thread_id` | `artifact_id` | Persistent TODOs (`TODO#LIST`) and Files (`FILE#<path>`) |

In [ ]:
from deep_agents_from_scratch.checkpoint_dynamo import (
    create_dynamodb_tables,
    wait_for_tables,
    get_checkpointer,
    DEFAULT_STATE_TABLE,
    DEFAULT_ARTIFACTS_TABLE,
)

# Create tables (idempotent — safe to run multiple times)
results = create_dynamodb_tables(region_name="us-east-1")
print("Table creation results:")
for table, status in results.items():
    print(f"  {table}: {status}")

# Wait for tables to be ready
wait_for_tables(region_name="us-east-1")

In [ ]:
# Get the checkpointer (auto-detects official package or uses fallback)
checkpointer = get_checkpointer(
    table_name=DEFAULT_STATE_TABLE,
    region_name="us-east-1",
)
print(f"Checkpointer type: {type(checkpointer).__name__}")

---

## Part 5: Model & All Imports

In [ ]:
from datetime import datetime

from langchain_aws import ChatBedrockConverse
from utils import format_messages, show_prompt, stream_agent

# --- Model ---
llm_nova_lite = ChatBedrockConverse(
    model="us.amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=0.0,
)

llm_nova_pro = ChatBedrockConverse(
    model="us.amazon.nova-2-pro-v1:0",
    region_name="us-east-1",
    temperature=0.0,
)

# Choose model (lite for testing, pro for production)
model = llm_nova_lite
print(f"✅ Model: {model.model}")

In [ ]:
# --- State ---
from deep_agents_from_scratch.state import DeepAgentState, Todo

# --- Persistent TODO Tools (DynamoDB) ---
from deep_agents_from_scratch.dynamo_tools import (
    dynamo_write_todos,
    dynamo_read_todos,
    dynamo_write_file,
    dynamo_read_file,
    dynamo_ls,
)

# --- Enhanced File Tools (in-memory, operate on state["files"]) ---
from deep_agents_from_scratch.enhanced_file_tools import edit_file, glob_files, grep_files

# --- Research Tools ---
from deep_agents_from_scratch.research_tools import tavily_search, think_tool

# --- Skills ---
from deep_agents_from_scratch.skills import (
    load_skill,
    discover_skills,
    get_skills_system_prompt,
    RESEARCH_SKILL_MD,
    CODE_REVIEW_SKILL_MD,
)

# --- Sub-agent construction ---
from deep_agents_from_scratch.task_tool import _create_task_tool, SubAgent

# --- Prompts ---
from deep_agents_from_scratch.prompts import (
    RESEARCHER_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS,
    FILE_USAGE_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
)

# --- Prompt Caching ---
from deep_agents_from_scratch.prompt_caching import build_cached_prompt

print("✅ All components imported successfully")

---

## Part 6: Review All Tools

### Persistent Tools (DynamoDB-backed)
These tools write to both in-memory state AND DynamoDB for cross-session persistence.

In [ ]:
print("=" * 60)
print("📦 PERSISTENT TOOLS (DynamoDB)")
print("=" * 60)
for t in [dynamo_write_todos, dynamo_read_todos, dynamo_ls, dynamo_read_file, dynamo_write_file]:
    print(f"\n🔧 {t.name}")
    print(f"   {t.description[:100]}...")

### Enhanced File Tools (in-memory)

These tools operate on `state["files"]` only. Since `dynamo_write_file` already syncs 
files to both memory and DynamoDB, these tools can search/edit the in-memory copy,
and any modifications are available for the persistent tools to read.

In [ ]:
print("=" * 60)
print("🔍 ENHANCED FILE TOOLS (in-memory)")
print("=" * 60)
for t in [edit_file, glob_files, grep_files]:
    print(f"\n🔧 {t.name}")
    print(f"   {t.description[:100]}...")

### Skills Tool

In [ ]:
print("=" * 60)
print("📋 SKILLS")
print("=" * 60)
print(f"\n🔧 {load_skill.name}")
print(f"   {load_skill.description[:150]}...")

# Preview example skills
from deep_agents_from_scratch.skills import parse_skill_md

for skill_md, label in [(RESEARCH_SKILL_MD, "web-research"), (CODE_REVIEW_SKILL_MD, "code-review")]:
    parsed = parse_skill_md(skill_md)
    print(f"\n   📋 {parsed['name']}: {parsed['description'][:70]}...")
    print(f"      Instructions: {len(parsed['instructions'])} chars (loaded on demand)")

---

## Part 7: Build the Production Agent

This is the single, unified agent construction — combining:
- **NB4's** base pattern (`create_agent` + sub-agents)
- **NB5's** enhanced tools and skills
- **NB6's** DynamoDB persistence and prompt caching

In [ ]:
# --- All orchestrator tools ---
orchestrator_tools = [
    # Planning (DynamoDB-persistent)
    dynamo_write_todos,
    dynamo_read_todos,
    # Files: persistent (DynamoDB + memory)
    dynamo_ls,
    dynamo_read_file,
    dynamo_write_file,
    # Files: enhanced (in-memory, works on synced state)
    edit_file,
    glob_files,
    grep_files,
    # Skills
    load_skill,
    # Thinking
    think_tool,
]

# --- Sub-agents (from NB4) ---
subagents = [
    SubAgent(
        name="research-agent",
        description="Delegated research agent for complex web searches. Has tavily_search and think_tool.",
        prompt=RESEARCHER_INSTRUCTIONS.format(date=datetime.now().strftime("%a %b %-d, %Y")),
        tools=["tavily_search", "think_tool"],
    ),
]

# --- Create task delegation tool ---
all_tools_for_registry = orchestrator_tools + [tavily_search]
task_tool = _create_task_tool(
    tools=all_tools_for_registry,
    subagents=subagents,
    model=model,
    state_schema=DeepAgentState,
)

# Final tool list
final_tools = orchestrator_tools + [task_tool]

print(f"✅ Orchestrator: {len(final_tools)} tools")
for t in final_tools:
    print(f"   🔧 {t.name}")
print(f"\n✅ Sub-agents: {len(subagents)}")
for sa in subagents:
    print(f"   🤖 {sa.name}: {sa.description[:60]}...")

In [ ]:
# --- Build Cached System Prompt ---
# Order: static (cached after Turn 1) → cache point → dynamic (re-processed each turn)

BASE_INSTRUCTIONS = """You are a highly capable AI assistant with planning, research, file management, and skills capabilities.

You operate in PERSISTENT MODE:
- Your TODOs are saved to DynamoDB via dynamo_write_todos / dynamo_read_todos
- Your files are saved to DynamoDB via dynamo_write_file / dynamo_read_file / dynamo_ls  
- Your conversation history persists across sessions via thread_id
- Always pass the thread_id parameter when using dynamo_* tools

For file editing (edit_file, glob_files, grep_files), these operate on the in-memory 
copy of files. Use dynamo_write_file first to create files, then use these tools to 
search and edit them."""

TOOL_INSTRUCTIONS = f"""{TODO_USAGE_INSTRUCTIONS}

{FILE_USAGE_INSTRUCTIONS}

{SUBAGENT_USAGE_INSTRUCTIONS.format(max_concurrent_research_units=3, max_researcher_iterations=3)}

## Skills System
You have access to a skills system. Skills are specialized instruction sets loaded from SKILL.md files.
- Use dynamo_ls() to discover available skills in the filesystem
- Use load_skill(name) to read full instructions when a skill is relevant
- Skills provide step-by-step guidance for specific tasks (research, code review, etc.)"""

DYNAMIC_CONTEXT = f"Today's date is {datetime.now().strftime('%A %B %-d, %Y')}."

# Build with optimal caching structure
cached_system_msg = build_cached_prompt(
    base_instructions=BASE_INSTRUCTIONS,
    tool_usage_instructions=TOOL_INSTRUCTIONS,
    dynamic_context=DYNAMIC_CONTEXT,
)

print(f"✅ System prompt: {len(cached_system_msg.content)} chars")
print(f"   Cache point: {cached_system_msg.additional_kwargs}")
print(f"   → Static prefix cached after Turn 1 (90% cost savings)")

In [ ]:
from langchain.agents import create_agent

# --- Create the Unified Production Agent ---
agent = create_agent(
    model,
    system_prompt=cached_system_msg.content,
    tools=final_tools,
    state_schema=DeepAgentState,
    checkpointer=checkpointer,  # DynamoDB state persistence!
)

print("✅ Production agent created!")
print(f"   Model:        {model.model}")
print(f"   Tools:        {len(final_tools)}")
print(f"   Sub-agents:   {len(subagents)}")
print(f"   Checkpointer: {type(checkpointer).__name__}")
print(f"   Cache:        Prompt caching enabled")

In [ ]:
# Visualize the agent graph
from IPython.display import Image, display
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

---

## Part 8: Pre-load Skills into the Virtual Filesystem

We seed the virtual filesystem with SKILL.md files. The agent discovers them via `dynamo_ls()` 
and loads them on-demand via `load_skill()`.

In [ ]:
# Pre-load skills into the virtual filesystem
initial_files = {
    "/skills/web-research/SKILL.md": RESEARCH_SKILL_MD,
    "/skills/code-review/SKILL.md": CODE_REVIEW_SKILL_MD,
}

# Verify skill discovery works
discovered = discover_skills(initial_files)
print("📋 Discovered Skills:")
for s in discovered:
    print(f"   • {s['name']}: {s['description'][:80]}...")

# Show the skills system prompt (progressive disclosure)
skills_prompt = get_skills_system_prompt(initial_files)
print(f"\n📝 Skills Prompt ({len(skills_prompt)} chars):")
print(skills_prompt)

---

## Part 9: Multi-Turn Test with Persistence

This demonstrates the key production features:
1. **Turn 1**: Agent receives a research request, creates TODOs, delegates research
2. **Turn 2**: Agent remembers context from Turn 1 (via DynamoDB checkpointer)
3. Files and TODOs persist in DynamoDB across both turns

In [ ]:
import uuid

# Create a unique thread for this session
thread_id = f"session_{uuid.uuid4().hex[:8]}"
config = {"configurable": {"thread_id": thread_id}}

print(f"🧵 Thread ID: {thread_id}")
print(f"   Reuse this ID to resume the conversation later!")

In [ ]:
# --- Turn 1: Research request ---
query_1 = "Give me an overview of Model Context Protocol (MCP)."

result_1 = await stream_agent(
    agent,
    {
        "messages": [{"role": "user", "content": query_1}],
        "files": initial_files,  # Pre-load skills
    },
    config=config,
)

In [ ]:
# View Turn 1 results
print("=" * 60)
print("TURN 1 RESPONSE")
print("=" * 60)
print(result_1["messages"][-1].content[:1000])

print(f"\n📁 Files: {list(result_1.get('files', {}).keys())[:5]}")

print("\n📋 TODOs:")
for todo in result_1.get("todos", []):
    status_emoji = {"pending": "⏳", "in_progress": "🔄", "completed": "✅"}
    emoji = status_emoji.get(todo["status"], "❓")
    print(f"   {emoji} {todo['content']} ({todo['status']})")

In [ ]:
# --- Turn 2: Follow-up (agent remembers Turn 1 context) ---
query_2 = "Based on your research, what are the main security concerns with MCP?"

result_2 = await stream_agent(
    agent,
    {"messages": [{"role": "user", "content": query_2}]},
    config=config,  # Same thread_id → same conversation!
)

In [ ]:
# View Turn 2 results
print("=" * 60)
print("TURN 2 RESPONSE (uses Turn 1 context via DynamoDB)")
print("=" * 60)
print(result_2["messages"][-1].content[:1000])

---

## Part 10: Verify DynamoDB Persistence

Let's query DynamoDB directly to confirm everything was persisted.

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource("dynamodb", region_name="us-east-1")

# Check State table (checkpoints)
state_table = dynamodb.Table("DeepAgents_State")
state_response = state_table.query(
    KeyConditionExpression=Key("pk").eq(thread_id),
    Limit=5,
)
print(f"📊 Checkpoints for thread '{thread_id}': {state_response['Count']}")

# Check Artifacts table (TODOs + Files)
artifacts_table = dynamodb.Table("DeepAgents_Artifacts")
artifacts_response = artifacts_table.query(
    KeyConditionExpression=Key("thread_id").eq(thread_id),
)
print(f"\n📊 Artifacts for thread '{thread_id}':")
for item in artifacts_response.get("Items", []):
    artifact_id = item["artifact_id"]
    content_preview = item.get("content", "")[:80]
    updated = item.get("updated_at", "?")
    print(f"   {artifact_id} (updated: {updated})")
    print(f"      {content_preview}...")

---

## Part 11: Test Enhanced File Tools

These tools operate on the in-memory `state["files"]`, which is synced with DynamoDB
via `dynamo_write_file`. Let's verify they work correctly.

In [ ]:
# Quick manual test of enhanced file tools
from deep_agents_from_scratch.enhanced_file_tools import edit_file, glob_files, grep_files

# Simulate a state with test files
test_state = {
    "files": {
        "notes.md": "# Notes\n\nHello World\nThis is a test.",
        "findings_mcp.md": "# MCP Findings\n\nMCP is a protocol for AI tool communication.",
        "findings_rag.md": "# RAG Findings\n\nRAG is retrieval augmented generation.",
        "readme.txt": "Just a readme file.",
    },
    "messages": [],
}

# Test glob_files (find by pattern)
print("🔍 glob_files('findings_*'):")
glob_result = glob_files.invoke(
    {"pattern": "findings_*"}, 
    config={"configurable": {"state": test_state}}
)
print(f"   {glob_result}")

# Test grep_files (search text)
print("\n🔍 grep_files('protocol'):")
grep_result = grep_files.invoke(
    {"pattern": "protocol"}, 
    config={"configurable": {"state": test_state}}
)
print(f"   {grep_result}")

print("\n✅ Enhanced file tools working correctly!")

---

## Part 12: Complete Architecture Summary

### What This Agent Has

| Category | Tool | Backend | From |
|----------|------|---------|------|
| **Planning** | `dynamo_write_todos` | DynamoDB | NB6 |
| **Planning** | `dynamo_read_todos` | DynamoDB | NB6 |
| **Files** | `dynamo_ls` | DynamoDB + Memory | NB6 |
| **Files** | `dynamo_read_file` | DynamoDB + Memory | NB6 |
| **Files** | `dynamo_write_file` | DynamoDB + Memory | NB6 |
| **Files** | `edit_file` | Memory | NB5 |
| **Files** | `glob_files` | Memory | NB5 |
| **Files** | `grep_files` | Memory | NB5 |
| **Skills** | `load_skill` | Memory | NB5 |
| **Research** | `tavily_search` | Web API | NB4 |
| **Research** | `think_tool` | None | NB4 |
| **Delegation** | `task` | — | NB4 |

### Production Features

| Feature | Implementation | Savings |
|---------|---------------|---------|
| **Prompt Caching** | `build_cached_prompt()` + `cachePoint` | 90% cost, 85% latency |
| **State Persistence** | DynamoDB checkpointer | Resume conversations |
| **Artifact Persistence** | `dynamo_*` tools | TODOs + Files survive restarts |
| **Skills** | SKILL.md + `load_skill` | Progressive disclosure |
| **Context Isolation** | `task` tool (sub-agents) | Clean research contexts |

### What the Library Has That We Don't (Yet)

- `execute` tool (sandboxed command execution)
- `PatchToolCallsMiddleware` (fixing orphaned tool calls)
- `SummarizationMiddleware` (compressing long conversation contexts)
- Multiple filesystem backends (disk, store, composite)